In [1]:
import requests

In [2]:
URL = 'https://nvsc.lrv.lt/lt/naujienos/covid-19-atveju-epidemiologinio-tyrimo-rezultatai'
page = requests.get(URL)

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
[strong.getText() for strong in soup.findAll('strong')]

['Jei jūs lankėtės pažymėtose vietose nurodytu laiku ir su jumis nesusisiekė NVSC specialistai, prašome užsiregistruoti paskambinus šiuo telefono numeriu: +370 62077547. Kreiptis tik darbo laiku (I-IV 8:00-17:00, V 8:00-15:45).',
 'Rekomendacijos dėl aplinkos valymo ir dezinfekcijos įmonėms, kuriose dirbo, gyveno, lankėsi COVID-19 susirgę asmenys \xa0 ',
 'Patvirtintų atvejų sąrašas:\xa0 ',
 '1. Patvirtintas 2020-02-28 (Šiauliai)',
 '2-3. Patvirtinti 2 atvejai 2020-03-10 (Kaunas)',
 '4.\xa0\xa0\xa0 Patvirtintas 2020-03-13 (Kaunas)',
 '5.\xa0\xa0\xa0 Patvirtintas 2020-03-13 (Klaipėda)',
 '6.\xa0\xa0\xa0 Patvirtintas: 2020-03-13 (Vilniaus m.)',
 '7.\xa0\xa0\xa0 Patvirtintas 2020-03-14 (Kretinga)',
 '8.\xa0\xa0\xa0 Patvirtintas 2020-03-14 (Klaipėda)',
 '9.\xa0\xa0\xa0 Patvirtintas 2020-03-14 (Vilniaus m.)',
 '10.\xa0\xa0\xa0 Patvirtintas 2020-03-14 (Panevėžys)',
 '11.\xa0\xa0\xa0 Patvirtintas 2020-03-15 d. (Vilniaus r.)',
 '12.\xa0\xa0\xa0 Patvirtintas 2020-03-15 (Vilniaus m.)',
 '13.\xa0

In [6]:
all_text = soup.getText()

In [7]:
import re

In [8]:
required_text = all_text[all_text.find('1. Patvirtintas'):]

In [9]:
required_text

'1. Patvirtintas 2020-02-28 (Šiauliai)\r\n1.\xa0\xa0\xa0 Asmuo 2020 m. vasario 24 d. 15:30 val. grįžo į Kauno oro uostą (KUN).\r\n2.\xa0\xa0\xa0 Taikyta atvejo saviizoliacija, 2020 m. vasario 25 d. nuotoliniu būdu išduotas nedarbingumo pažymėjimas, pasireiškė lengvi klinikiniai ūminės viršutinių kvėpavimo takų infekcijos simptomai.\r\n3.\xa0\xa0\xa0 2020 m. vasario 27 d. kreipėsi į VšĮ Respublikinės Šiaulių ligoninės Suaugusiųjų infekcinių ligų skyrių, 2020 vasario 28 d. patvirtinta COVID-19 diagnozė.\r\n4.\xa0\xa0\xa0 Kovo 14 d. pranešta, kad asmuo išleistas namo.\n\n2-3. Patvirtinti 2 atvejai 2020-03-10 (Kaunas)\r\n1.\xa0\xa0\xa0 Sutuoktiniai 2020 m. kovo 7 d. ryte grįžo iš Šiaurės Italijos lengvuoju automobiliu namo, į Kauną.\r\n2.\xa0\xa0\xa0 2020 m. kovo 7 d. vakare, apie 18 val., dalyvavo pobūvyje restorane, adresu S. Daukanto g. 23, Kaunas. AKTUALU IKI: kovo 22 d.\r\n3.\xa0\xa0 \xa02020 m. kovo 9 d. lankėsi Kauno Akropolyje.\r\n4.\xa0\xa0\xa0 2020 m. kovo 9 d. vyras pasijuto sus

In [10]:
indexes_of_start = [m.start() for m in re.finditer('Patvirtin', required_text)]

In [11]:
re.findall('\d+', 'asdf 43 3. ')

['43', '3']

In [12]:
cases = []
case_numbers_all = []
for start, end in zip(indexes_of_start[:-1], indexes_of_start[1:]):
    case_numbers = re.findall('\d+', required_text[start-10:start])
    case_number = case_numbers[-1] if case_numbers else -1
    cases.append(required_text[start:end].strip().replace(u'\xa0', u' '))
    case_numbers_all.append(case_number)
start = indexes_of_start[-1]
end = required_text[start:].find('Dalintis')
case_numbers = re.findall('\d+', required_text[start-10:start])
case_number = case_numbers[-1] if case_numbers else -1
cases.append(required_text[start:end+start].strip().replace(u'\xa0', u' '))
case_numbers_all.append(case_number)

In [13]:
cases[-1]

'Patvirtinti 2 atvejai 2020-04-03 (Kaunas) \n1. Asmuo 2020-03-22 19 val. grįžo iš Paryžiaus į Vilniaus oro uostą X99603.\n2. 2020-03-22 iš oro uosto grįžo City Bee automobiliu.\n3. 2020-04-02 susirgo.\n4. 2020-04-02 mobiliajame punkte atliko tyrimą, važiavo kartu nuosavu automobiliu.'

In [14]:
def get_text_between_parentheses(s):
    if s.find("(") != -1 and s.find(")") != -1:
        return s[s.find("(")+1:s.find(")")]
    else:
        return ""

In [15]:
'213.34'.split(':')

['213.34']

In [16]:
def extract_time_from_string(s):
    rez = re.findall(r'([0-2]?[0-9][:|.|,][0-5][0-9])|([0-2][0-9] val?.(\s[0-5][0-9] min.)?)', s)
    times = [max(match, key=len) for match in rez]
    correct_formats = []
    for time in times:
        digits = re.findall('\\d+', time)
        if len(digits) == 1:
            hour = digits[0]
            hour = f'0{hour}' if len(hour) == 1 else hour
            correct_formats.append(f'{hour}:00')
        else:
            hour = digits[0]
            hour = f'0{hour}' if len(hour) == 1 else hour
            correct_formats.append(f'{hour}:{digits[1]}')
    #correct_format = [':'.join() for time in times]
    return correct_formats

In [17]:
re.findall('\\d+', '19:30')

['19', '30']

In [18]:
rez = re.findall(r'([0-2]?[0-9][:|.|,][0-5][0-9])|([0-2][0-9] val.\s?([0-5][0-9] min.))', 'Buvome 15 val. 30 min. KIne ir 19:20 19:30 temp dat')

In [19]:
[max(match, key=len) for match in rez]

['15 val. 30 min.', '19:20', '19:30']

In [20]:
def get_text_between_quotations(s):
    s = s.replace(",,", "„").replace("“", "\"")
    start = s.find("„")
    end = s.find("\"")
    if start != -1 and end != -1:
        return s[start+1:end]
    else:
        return ""

In [21]:
def get_airport_address_if_present(s):
    airport_addresses = {'Vilniaus': 'Rodūnios kl. 10A', 'Kauno': 'Oro uosto g. 4', 'Palangos': 'Liepojos pl. 1'}
    airport_city = {'Vilniaus': 'Vilnius', 'Kauno': 'Kaunas', 'Palangos': 'Palanga'}
    address_to_return = None
    name_to_return = None
    city_to_return = None
    if ' oro ' in s:
        for city, address in airport_addresses.items():
            if city.lower() in s.lower():
                address_to_return = address
                name_to_return = f"{city} oro uostas"
                city_to_return = airport_city[city]
    return address_to_return, name_to_return, city_to_return

In [51]:
def get_sea_port_if_present(s):
    if "Klaipėdos jūrų" in s:
        return "Baltijos pr. 40", "Klaipėdos jūrų uostas", "Klaipėda"
    return None, None, None

In [22]:
test = '1. Asmuo 2020-03-13  10.30 val. lėktuvu (reisas: TK 181 skrido  iš Meksikos (Kankuno oro uostas),'\
'persėdimas Stambule. Iš Stambulo (Stambulo oro uostas) (reisas TK 1407 ) grįžo į Vilniaus oro uostą,\n'\
'2.2020-03-13 Iš Vilniaus oro uosto nuosavu automobiliu su šeimos nariais grįžo į  namus (Joniškio r.).\n '\
' 3. 2020-03-14 16 val. lankėsi parduotuvėje ,,Norfa" (Vilniaus g. 47B, Joniškis).'\


In [23]:
test_case = 'Patvirtintas 2020-02-28 (Šiauliai)\r\n1.    Asmuo 2020 m. vasario 24 d. 15:30 val. grįžo į Kauno oro uostą (KUN).\r\n2.    Taikyta atvejo saviizoliacija, 2020 m. vasario 25 d. nuotoliniu būdu išduotas nedarbingumo pažymėjimas, pasireiškė lengvi klinikiniai ūminės viršutinių kvėpavimo takų infekcijos simptomai.\r\n3.    2020 m. vasario 27 d. kreipėsi į VšĮ Respublikinės Šiaulių ligoninės Suaugusiųjų infekcinių ligų skyrių, 2020 vasario 28 d. patvirtinta COVID-19 diagnozė.\r\n4.    Kovo 14 d. pranešta, kad asmuo išleistas namo.\n\n2-3.'

In [24]:
print(repr(test_case))

'Patvirtintas 2020-02-28 (Šiauliai)\r\n1.    Asmuo 2020 m. vasario 24 d. 15:30 val. grįžo į Kauno oro uostą (KUN).\r\n2.    Taikyta atvejo saviizoliacija, 2020 m. vasario 25 d. nuotoliniu būdu išduotas nedarbingumo pažymėjimas, pasireiškė lengvi klinikiniai ūminės viršutinių kvėpavimo takų infekcijos simptomai.\r\n3.    2020 m. vasario 27 d. kreipėsi į VšĮ Respublikinės Šiaulių ligoninės Suaugusiųjų infekcinių ligų skyrių, 2020 vasario 28 d. patvirtinta COVID-19 diagnozė.\r\n4.    Kovo 14 d. pranešta, kad asmuo išleistas namo.\n\n2-3.'


In [25]:
def extract_actions_from_case(case):
    actions_by_case = []
    matches = list(re.finditer('(^|\n)\s*(\d\.)', case))
    if not matches:
        return []
    for m1, m2 in zip(matches[:-1], matches[1:]):
        start_idx = m1.start(0)
        end_idx = m2.start(0)
        actions_by_case.append(case[start_idx:end_idx].strip().replace('\n', ' '))
    actions_by_case.append(case[matches[-1].start():].strip().replace('\n', ' '))
    return actions_by_case

In [26]:
extract_actions_from_case(test_case)

['1.    Asmuo 2020 m. vasario 24 d. 15:30 val. grįžo į Kauno oro uostą (KUN).',
 '2.    Taikyta atvejo saviizoliacija, 2020 m. vasario 25 d. nuotoliniu būdu išduotas nedarbingumo pažymėjimas, pasireiškė lengvi klinikiniai ūminės viršutinių kvėpavimo takų infekcijos simptomai.',
 '3.    2020 m. vasario 27 d. kreipėsi į VšĮ Respublikinės Šiaulių ligoninės Suaugusiųjų infekcinių ligų skyrių, 2020 vasario 28 d. patvirtinta COVID-19 diagnozė.',
 '4.    Kovo 14 d. pranešta, kad asmuo išleistas namo.  2-3.']

In [27]:
def extract_lithuanian_date(s):
    months = {'vasar': '02', 'kov':'03', 'balan':'04'}
    '''
    start = s.find('2020')
    end = s.find(' d.')
    if start != -1 and end != -1:
        date = s[start-1:end+3]
        return date
    '''
    existing_months = [month for month in months.keys() if month in s]
    if existing_months:
        start = s.find(existing_months[0])
        end = s[start:].find(' d.')+start
        if start != -1 and end != -1:
            day = re.findall(r'\d+', s[start-5:end+3])[-1]
            date = f'2020-{months[existing_months[0]]}-{day}'
            return date
    return '-'
    

In [28]:
def check_if_possible_address(s):
    address_abbrev = [' g.', ' pl.', ' pr.', ' al.']
    if [abbr for abbr in address_abbrev if abbr in s]:
        return True
    else:
        return False

In [29]:
def extract_street_addresses(s):
    address_abbrev = [' g\.', ' pl\.', ' pr\.', ' al\.', ' akl\.']
    found_addresses = []
    found_cities = []
    s = s.replace('(', ' ').replace(')', ' ')
    for abbrv in address_abbrev:
        all_locations = [m.start() for m in re.finditer(abbrv, s)]
        for indx in all_locations:
            all_words_before = [i for i in s[:indx].split(' ') if i]
            all_words_after = [i for i in s[indx+len(abbrv)-1:].split(' ') if i]
            
            address_words = []
            for word in reversed(all_words_before):
                word = word.replace("(", "")
                if len(word) <= 3 and 'ir' in word:
                    address_words.append(word)
                    continue
                if word and word[0].isupper():
                    address_words.append(word)
                else:
                    break
            
            address_words = list(reversed(address_words))
            address_words.append(abbrv.replace('\\', ''))
            if all_words_after and all_words_after[0][0].isdigit():
                address_words.append(all_words_after[0])
            found_addresses.append(" ".join(address_words).replace(',', ''))
            
            city_words = []
            city_abbrv_added = False
            if len(all_words_after) >= 2:
                for word in all_words_after[1:]:
                    if 1 < len(word) <= 3:
                        city_words.append(word)
                        city_abbrv_added = True
                        continue
                    if word and word[0].isupper() and not city_abbrv_added:
                        city_words.append(word)
                    else:
                        break
            found_cities.append(" ".join(city_words))
            
    return found_addresses, found_cities

In [30]:
test_string_address = "2020-03-17 pasireiškus klinikiniams simptomams, 19 val. iš namų (Šiauliai)  GMP automobiliu nuvežtas į Respublikinę Šiaulių ligoninę (V. Kudirkos g. 99, Šiauliai), atlikus laboratorinius tyrimus Dariaus ir Girėno g. 5E parvežtas ta patį vakarą į namus (Šiauliai)."
test_string_address_2 = '3. 2020-03-16 – 2020-03-18 lankėsi darbe (Livonijos  g.15, Joniškis).'

In [31]:
extract_street_addresses(test_string_address_2)

(['Livonijos\xa0  g. 15'], ['Joniškis'])

In [32]:
extract_street_addresses('3. 2020-03-30 nuosavu automobiliu nuvyko į mobilųjį punktą dėl laboratorinių tyrimų atlikimo (Alyvų g. 8, Biržų k.). Veliau lankesi kazkur')

(['Alyvų  g. 8'], ['Biržų k.'])

In [33]:
def extract_date(s):
    if re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", s):
        return re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", s).group()
    else:
        return extract_lithuanian_date(s)

In [34]:
extract_actions_from_case(test_case)

['1.    Asmuo 2020 m. vasario 24 d. 15:30 val. grįžo į Kauno oro uostą (KUN).',
 '2.    Taikyta atvejo saviizoliacija, 2020 m. vasario 25 d. nuotoliniu būdu išduotas nedarbingumo pažymėjimas, pasireiškė lengvi klinikiniai ūminės viršutinių kvėpavimo takų infekcijos simptomai.',
 '3.    2020 m. vasario 27 d. kreipėsi į VšĮ Respublikinės Šiaulių ligoninės Suaugusiųjų infekcinių ligų skyrių, 2020 vasario 28 d. patvirtinta COVID-19 diagnozė.',
 '4.    Kovo 14 d. pranešta, kad asmuo išleistas namo.  2-3.']

In [52]:
action_addresses = []
action_place_names = []
all_actions = []
action_dates = []
action_city = []
action_case_ids = []
action_times = []
for case_number, case in zip(case_numbers_all, cases):
    actions = extract_actions_from_case(case)
    case_has_airport = False
    for action in actions:
        found_address = False
        found_name = False
        if action.strip() == '':
            continue
        all_actions.append(action)
        parentheses_text_inside = get_text_between_parentheses(action)
        street_addresses, city = extract_street_addresses(action)
        if street_addresses:
            action_addresses.append('/'.join(street_addresses))
            action_city.append('/'.join(city))
            found_address = True
        else:
            action_addresses.append("-")
            action_city.append("-")
        qoutation_text_inside = get_text_between_quotations(action)
        if qoutation_text_inside:
            action_place_names.append(qoutation_text_inside)
            found_name = True
        else:
            action_place_names.append("-")
        if not found_address and not found_name and not case_has_airport:
            airport_addr, airport_name, airport_city = get_airport_address_if_present(action)
            if airport_addr:
                action_addresses[-1] = airport_addr
                action_place_names[-1] = airport_name
                action_city[-1] = airport_city
                case_has_airport = True
            port_addr, port_name, port_city = get_sea_port_if_present(action)
            if port_addr:
                action_addresses[-1] = port_addr
                action_place_names[-1] = port_name
                action_city[-1] = port_city
                case_has_airport = True
        print(extract_time_from_string(action))
        action_time = '/'.join(extract_time_from_string(action))
        
        action_times.append(action_time)
        action_dates.append(extract_date(action))
        action_case_ids.append(case_number)

['15:30']
[]
['20:00']
[]
[]
['18:00']
[]
[]
[]
[]
[]
[]
['20:00']
[]
[]
['00:10']
[]
[]
[]
['14:20']
['18:00']
[]
['17:37', '17:59']
[]
['18:00']
[]
[]
[]
['14:20']
['16:00']
['14:50']
[]
[]
[]
['20:40']
[]
[]
[]
[]
[]
[]
[]
['13:30']
[]
['14:00']
['15:00']
['18:00']
['11:30']
['04:25']
['23:00']
[]
['17:00']
[]
['23:30']
[]
[]
[]
[]
[]
[]
[]
['23:35']
[]
['12:57', '13:57']
[]
['21:27']
[]
['22:00']
[]
['14:25']
['18:30', '21:30']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['16:00']
[]
[]
[]
[]
[]
['14:20']
['16:00']
['14:50']
[]
[]
['12:10']
['14:00']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['17:00']
[]
[]
[]
[]
[]
[]
[]
[]
['20:00']
['22:00']
[]
['19:00']
[]
[]
['15:30']
[]
[]
['03:20', '14:00']
['04:20']
['05:20', '14:30']
['06:20']
['08:30', '11:00', '14:00']
[]
[]
[]
['15:00']
[]
[]
[]
[]
[]
[]
[]
['14:00']
['15:00', '15:30']
[]
['15:25']
[]
[]
['18:00']
['18:00']
[]
['20:10']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['23:35']
[]
[]
[]
['13:15']
[]
[]
[]
[]
[]
[]
['10:30']
[]
[]
['15:00']
[]
[]
[]


In [53]:
import pandas as pd

In [54]:
df = pd.DataFrame({'case_id': action_case_ids, 'actions': all_actions, 'addresses': action_addresses, 'action_place_names': action_place_names, "city": action_city, "dates": action_dates, "time": action_times})

In [55]:
boring_data_contains = ['temperatūra']

In [56]:
df = df[~df['actions'].str.contains('Asmuo niekur nebuvo')]
df = df[~df['actions'].str.contains('izolia')]
df = df[~df['actions'].str.contains('kreipėsi')]
df = df[~df['actions'].str.contains('temperatūr')].reset_index(drop=True)

In [65]:
df = df[(df['addresses'] != '-') | (df['action_place_names'] != '-')]

In [69]:
df[pd.to_numeric(df['case_id']) > 211].to_csv('extracted_cases.csv', encoding='utf-8-sig', index=False)

In [70]:
df[pd.to_numeric(df['case_id']) > 211]

,case_id,actions,addresses,action_place_names,city,dates,time
605,212,1.Asmuo 2020-02-19 per Kauno oro uostą išvyko ...,Oro uosto g. 4,Kauno oro uostas,Kaunas,2020-02-19,
608,213,1.Asmuo 2020-03-14 20 val. lėktuvu (reisas: 3...,Oro uosto g. 4,Kauno oro uostas,Kaunas,2020-03-14,20:00
611,213,4. 2020-03-17 pasireiškus klinikiniams simptom...,V. Kudirkos g. 99,-,Šiauliai,2020-03-17,19:00
614,215,1. Asmuo 2020-03-13 10.30 val. lėktuvu (reisa...,Rodūnios kl. 10A,Vilniaus oro uostas,Vilnius,2020-03-13,10:30
616,215,"2. 2020-03-14 16 val. lankėsi parduotuvėje ,,N...",Vilniaus g. 47B,Norfa,Joniškis,2020-03-14,16:00
...,...,...,...,...,...,...,...
753,294,3. 2020-03-30 nuosavu automobiliu nuvyko į mob...,J. Jablonskio g. 16,-,Šiauliai,2020-03-30,
754,295,1. Asmuo 2020-03-14 grįžo iš Lutono į Kauno or...,Oro uosto g. 4,Kauno oro uostas,Kaunas,2020-03-14,
758,296,1. Asmuo 2020-03-19 grįžo iš Frankfurto į Viln...,Rodūnios kl. 10A,Vilniaus oro uostas,Vilnius,2020-03-19,
762,297,1. Asmuo 2020-03-19 grįžo iš Frankfurto į Viln...,Rodūnios kl. 10A,Vilniaus oro uostas,Vilnius,2020-03-19,
